In [2]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

# Author: Milan Ondrasovic <milan.ondrasovic@gmail.com>

from typing import *

import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms

import cv2 as cv
import numpy as np

import matplotlib.pyplot as plt

from PIL import Image, ImageStat, ImageOps

from sot.bbox import BBox
from sot.utils import Size

plt.style.use('tableau-colorblind10')
plt.rcParams['font.family'] = 'Times New Roman'
plt.rcParams['font.size'] = 11

%load_ext autoreload
%autoreload 2

In [14]:
my_model_file_path = "../model_49.pth"
my_model = torch.load(my_model_file_path, map_location=torch.device('cpu'))

new_model_file_path = "../siamfc_alexnet_e50.pth"
new_model = torch.load(new_model_file_path, map_location=torch.device('cpu'))

(35, 30)

In [17]:
def print_model_info(model):
    for item in model.items():
        print(f"{item[0]}: {list(item[1].shape)}")

print_model_info(my_model)

conv1.block.0.weight: [96, 3, 11, 11]
conv1.block.0.bias: [96]
conv1.block.1.weight: [96]
conv1.block.1.bias: [96]
conv1.block.1.running_mean: [96]
conv1.block.1.running_var: [96]
conv1.block.1.num_batches_tracked: []
conv2.block.0.weight: [256, 48, 5, 5]
conv2.block.0.bias: [256]
conv2.block.1.weight: [256]
conv2.block.1.bias: [256]
conv2.block.1.running_mean: [256]
conv2.block.1.running_var: [256]
conv2.block.1.num_batches_tracked: []
conv3.block.0.weight: [384, 256, 3, 3]
conv3.block.0.bias: [384]
conv3.block.1.weight: [384]
conv3.block.1.bias: [384]
conv3.block.1.running_mean: [384]
conv3.block.1.running_var: [384]
conv3.block.1.num_batches_tracked: []
conv4.block.0.weight: [384, 384, 3, 3]
conv4.block.0.bias: [384]
conv4.block.1.weight: [384]
conv4.block.1.bias: [384]
conv4.block.1.running_mean: [384]
conv4.block.1.running_var: [384]
conv4.block.1.num_batches_tracked: []
conv5.block.0.weight: [256, 192, 3, 3]
conv5.block.0.bias: [256]
conv5.block.1.weight: [256]
conv5.block.1.bias